In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
symbol = ["AAPL"]
start_date = "2007-01-02"
features = yf.download(symbol, start=start_date)

In [ ]:
# Drop multiple columns
features = features.drop(['Open', 'High','Low', 'Adj Close','Volume'], axis=1)

In [ ]:
features = features.dropna(axis=0)
features.isna().sum()

In [ ]:
df_plot = pd.DataFrame(features, index=features.index)
# Plot the DataFrame
df_plot.plot(figsize=(18,9))
plt.show()

In [ ]:
features.shape

In [ ]:
pip install pmdarima

In [ ]:
import pandas as pd
import numpy as np
df = features 
df.shape

In [ ]:
from statsmodels.tsa.stattools import adfuller

def ad_test(dataset):
  dftest = adfuller(dataset, autolag = "AIC")
  print("1. ADF : " , dftest[0])
  print("2. P-value :" , dftest[1])
  print("3. Num of Lags :" , dftest[2])
  print("4. Num of Observations Used For ADF Regressor and Critical Value Calculations  :" , dftest[3])
  print("5. Critical Values :")
  for key, val in dftest[4].items():
    print("\t", key,": ", val)

In [ ]:
ad_test(df["Close"])

In [ ]:
from pmdarima import auto_arima
import warnings
warnings.filterwarnings("ignore")

In [ ]:
stepwise_fit = auto_arima(df["Close"], trace=True, suppress_warnings = True, stepwise=False, seasonal=False)
stepwise_fit.summary()

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

In [ ]:
print(df.shape)
train = df.iloc[:-30]
test = df.iloc[-30:]
print(train.shape,test.shape)

In [ ]:
import statsmodels.api as sm
model = sm.tsa.arima.ARIMA(train["Close"], order=(4,1,1))
model = model.fit()
model.summary()

In [ ]:
start = len(train)
end = len(train)+len(test)-1
pred = model.predict(start=start,end=end,typ='levels')
pred.index = df.index[start:end+1]
print(pred)

In [ ]:
pred.plot(legend=True)
test["Close"].plot(legend=True)

In [ ]:
from sklearn.metrics import mean_squared_error 
from math import sqrt
rmse = sqrt(mean_squared_error(pred,test["Close"]))
print(rmse)

In [ ]:
test["Close"].mean()

In [ ]:
model2 = sm.tsa.arima.ARIMA(train["Close"], order=(4,1,1))
model2 = model2.fit()

In [ ]:
future_index=pd.date_range(start="2023-05-04", end = "2023-06-02")
pred = model2.predict(start = len(df),end=len(df)+29,typ='levels').rename('ARIMA Prdictions')
pred.index = future_index

In [ ]:
pred

In [ ]:
# Calculate RMSE, MAPE, and MAE
from sklearn.metrics import mean_squared_error, mean_absolute_error 
from math import sqrt
rmse = sqrt(mean_squared_error(pred,test["Close"]))
mape = np.mean(np.abs((test["Close"] - pred) / test["Close"])) * 100
mae = mean_absolute_error(pred, test["Close"])

# Print metrics
print(f'RMSE: {rmse:.2f}')
print(f'MAPE: {mape:.2f}%')
print(f'MAE: {mae:.2f}')